<a href="https://colab.research.google.com/github/NetoRibeiro/DATA8001Assignment1/blob/main/Linear_Regression_Model_01042021_009_ToFit_RStudio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Import and Load File

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
from matplotlib import colors
import numpy as np
import pandas as pd
from datetime import datetime as dt

import calendar
import re
import string

import math
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score

from scipy import stats

In [2]:
df_processed = pd.read_csv('/content/drive/MyDrive/Data Science and Analytics/R00206995/data/R00206995_processed.csv')

In [3]:
df_processed.head()

,car_reg,purchase_date,year,month,county,make,model,type,colour,tax_band,price
0,202-C-2315,2020-07-01,2020,7,CORK,AUDI,A4,SALOON,RED,B,55287.0
1,191-C-3750,2019-01-20,2019,1,CORK,MAZDA,CX-30,SUV,SILVER,B,41690.0
2,191-L-3155,2019-03-21,2019,3,LIMERICK,BMW,3 SERIES,SALOON,WHITE,C,40381.0
3,191-D-2645,2019-01-26,2019,1,DUBLIN,AUDI,Q3,SUV,ORANGE,C,44836.0
4,181-W-341,2018-01-27,2018,1,WATERFORD,OPEL,ASTRA,HATCHBACK,RED,A,32188.0


###Transformation and Split

In [4]:
def get_feature_stats(df, list_columns):
    lis_of_string_features = []
    for feature in list_columns:
      if df[feature].dtype not in ['uint8', 'int16','int32','int64','float16','float32','float64']:
        lis_of_string_features.append(feature)
    return lis_of_string_features


def concat_columns(col1, col2, delimited):
  try:
    cols = f'{col1}{delimited}{col2}'
    return cols
  except Exception as ex:
    raise Exception(f'Error when concatenate:: {ex}')


def apply_calc_columns(col1, col2, calc):
  try:
    col1 = float(col1)
    col2 = float(col2)
 
    if calc == 'times':
      cols = col1 * col2
 
    elif calc == 'div':
      try:
        cols == col1 / col2
      except Exception as ex:
        raise Exception(f'Error when try to divide:: {ex}')
        return 0.0
 
    elif calc == 'plus':
      cols = col1 + col2
 
    elif calc == 'minus':
      cols = col1 - col2
 
    else:
      cols = 0

    return cols
  except Exception as ex:
    raise Exception(f'Error when convert to a float:: {ex}')


In [5]:
df_concats = df_processed.copy()

In [6]:
#df_concats['makemodel'] = df_concats.apply(lambda row: concat_columns(row['make'], row['model'],"_"), axis=1)
#df_concats.head(3)

In [7]:
# Create a list of columns to apply Dummies

list_of_dum_features = ['tax_band', 'type']

# Apply Lower case before dummies
df_concats['type'] = df_concats['type'].str.lower()
df_concats['tax_band'] = df_concats['tax_band'].str.lower()
df_concats['make'] = df_concats['make'].str.lower()
df_concats['model'] = df_concats['model'].str.lower()
df_tofit = df_concats.copy()

# Apply Dammies
df_tofit = pd.get_dummies(df_concats, columns=list_of_dum_features)
df_tofit.drop(['car_reg', 'purchase_date', 'month'], axis=1, inplace=True)

In [43]:
list_of_col_not_use = []
for feature in list_of_dum_features:
  for col in df_tofit.columns:
    if (re.search(feature, col)):
      list_of_col_not_use.append(col)
list_of_col_not_use

['tax_band_a',
 'tax_band_b',
 'tax_band_c',
 'tax_band_d',
 'tax_band_e',
 'type_coupe',
 'type_estate',
 'type_hatchback',
 'type_saloon',
 'type_suv']

In [9]:
#list_columns = df_dummies.columns
list_columns = df_tofit.columns
list_new_columns = df_tofit.columns
#list_feature = get_feature_stats(df_dummies, list_columns)
list_feature = get_feature_stats(df_tofit, list_columns)
list_new_features = get_feature_stats(df_tofit, list_new_columns)

In [10]:
# df_train, df_test = train_test_split(df_dummies.copy() ,test_size=0.2, random_state=8001)
df_train, df_test = train_test_split(df_tofit.copy() ,test_size=0.2, random_state=8001)

In [11]:
df_tofit.head(3)

,year,county,make,model,colour,price,tax_band_a,tax_band_b,tax_band_c,tax_band_d,tax_band_e,type_coupe,type_estate,type_hatchback,type_saloon,type_suv
0,2020,CORK,audi,a4,RED,55287.0,0,1,0,0,0,0,0,0,1,0
1,2019,CORK,mazda,cx-30,SILVER,41690.0,0,1,0,0,0,0,0,0,0,1
2,2019,LIMERICK,bmw,3 series,WHITE,40381.0,0,0,1,0,0,0,0,0,1,0


In [12]:
df_train.head(3)

,year,county,make,model,colour,price,tax_band_a,tax_band_b,tax_band_c,tax_band_d,tax_band_e,type_coupe,type_estate,type_hatchback,type_saloon,type_suv
654,2018,DUBLIN,mazda,cx-30,WHITE,41920.0,0,0,1,0,0,0,0,0,0,1
1947,2018,WATERFORD,ford,kuga,WHITE,34961.0,0,1,0,0,0,0,0,0,0,1
3698,2020,DUBLIN,bmw,3 series,BLUE,69784.0,0,1,0,0,0,0,0,0,1,0


In [13]:
list_feature

['county', 'make', 'model', 'colour']

In [32]:
#Convert all string features to a int features
for feature in list_feature:
  label_condition = LabelEncoder()
  df_train[feature] = 1 + label_condition.fit_transform(df_train[feature])
  df_test[feature] = 1 + label_condition.fit_transform(df_test[feature])
  df_tofit[feature] = 1 + label_condition.fit_transform(df_tofit[feature])

In [33]:
list_of_col_use

['tax_band_a',
 'tax_band_b',
 'tax_band_c',
 'tax_band_d',
 'tax_band_e',
 'type_coupe',
 'type_estate',
 'type_hatchback',
 'type_saloon',
 'type_suv']

In [34]:
label_condition.classes_ 

array(['BLUE', 'ORANGE', 'RED', 'SILVER', 'WHITE'], dtype=object)

In [35]:
# df_tofit['county_cat'] = df_tofit['county_lbl'].astype('category')

In [36]:
# df_tofit['county_lbl_cat'] = df_tofit['county_cat'].cat.codes + 1

In [37]:
df_tofit.head(3)

,year,county,make,model,colour,price,tax_band_a,tax_band_b,tax_band_c,tax_band_d,tax_band_e,type_coupe,type_estate,type_hatchback,type_saloon,type_suv
0,2020,1,1,6,3,55287.0,1,2,1,1,1,1,1,1,2,1
1,2019,1,4,15,4,41690.0,1,2,1,1,1,1,1,1,1,2
2,2019,4,2,2,5,40381.0,1,1,2,1,1,1,1,1,2,1


In [38]:
df_train['make_model'] = df_train.apply(lambda row: apply_calc_columns(row['make'], row['model'],"times"), axis=1)
df_test['make_model'] = df_test.apply(lambda row: apply_calc_columns(row['make'], row['model'],"times"), axis=1)
df_tofit['make_model'] = df_tofit.apply(lambda row: apply_calc_columns(row['make'], row['model'],"times"), axis=1)

In [39]:
df_train['make_colour'] = df_train.apply(lambda row: apply_calc_columns(row['make'], row['colour'],"times"), axis=1)
df_test['make_colour'] = df_test.apply(lambda row: apply_calc_columns(row['make'], row['colour'],"times"), axis=1)
df_tofit['make_colour'] = df_tofit.apply(lambda row: apply_calc_columns(row['make'], row['colour'],"times"), axis=1)

In [40]:
df_train['make_model_colour'] = df_train.apply(lambda row: apply_calc_columns(row['make_model'], row['colour'],"times"), axis=1)
df_test['make_model_colour'] = df_test.apply(lambda row: apply_calc_columns(row['make_model'], row['colour'],"times"), axis=1)
df_tofit['make_model_colour'] = df_tofit.apply(lambda row: apply_calc_columns(row['make_model'], row['colour'],"times"), axis=1)

In [41]:
df_train['year_county'] = df_train.apply(lambda row: apply_calc_columns(row['year'], row['county'],"times"), axis=1)
df_test['year_county'] = df_test.apply(lambda row: apply_calc_columns(row['year'], row['county'],"times"), axis=1)
df_tofit['year_county'] = df_tofit.apply(lambda row: apply_calc_columns(row['year'], row['county'],"times"), axis=1)

In [42]:
df_tofit.to_csv('/content/drive/MyDrive/Data Science and Analytics/R00206995/data/R00206995_test_tofit.csv', index=False)

In [44]:
# X variables (independent)
output_feature_dummies = 'price'
input_features_new_fit = [item for item in df_tofit.columns if item not in [output_feature_dummies] + list_of_col_not_use]
print(input_features_new_fit)

['year', 'county', 'make', 'model', 'colour', 'make_model', 'make_colour', 'make_model_colour', 'year_county']


In [46]:
# X variables (independent)
output_feature_dummies = 'price'
input_features_dummies = [item for item in df_train.columns if item not in [output_feature_dummies] + list_of_col_not_use]
print(input_features_dummies)

['year', 'county', 'make', 'model', 'colour', 'make_model', 'make_colour', 'make_model_colour', 'year_county']


In [47]:
# Apply Standar Scaler for a list of features
for column in df_tofit.columns:

  if column in input_features_new_fit:
    scaler_column = StandardScaler()
    scaler_column.fit(df_tofit[column].values.reshape(-1, 1))
    df_tofit[column] = scaler_column.transform(df_tofit[column].values.reshape(-1, 1))
      
  else:
    print(f'skip:\t{column}')
df_tofit.to_csv('/content/drive/MyDrive/Data Science and Analytics/R00206995/data/R00206995_tofit.csv', index=False)

skip:	price
skip:	tax_band_a
skip:	tax_band_b
skip:	tax_band_c
skip:	tax_band_d
skip:	tax_band_e
skip:	type_coupe
skip:	type_estate
skip:	type_hatchback
skip:	type_saloon
skip:	type_suv


In [48]:
#df_dummies.head(3)

In [49]:
# Apply Standar Scaler for a list of features
for column in df_train.columns:

  if column in input_features_dummies:
    scaler_column = StandardScaler()
    scaler_column.fit(df_train[column].values.reshape(-1, 1))
    df_train[column] = scaler_column.transform(df_train[column].values.reshape(-1, 1))
    df_test[column] = scaler_column.transform(df_test[column].values.reshape(-1, 1))
    
  
  else:
    print(f'skip:\t{column}')
# df_dummies.to_csv('/content/drive/MyDrive/Data Science and Analytics/R00206995/data/R00206995_tofit_dum.csv', index=False)

skip:	price
skip:	tax_band_a
skip:	tax_band_b
skip:	tax_band_c
skip:	tax_band_d
skip:	tax_band_e
skip:	type_coupe
skip:	type_estate
skip:	type_hatchback
skip:	type_saloon
skip:	type_suv


###Linear Regressor Model
####Standard Scaler plus Best Features Dummies

In [50]:
X_train = df_train[input_features_dummies].values
X_test = df_test[input_features_dummies].values

y_train = df_train[[output_feature_dummies]].values
y_test = df_test[[output_feature_dummies]].values

In [51]:
df_train.head(3)

,year,county,make,model,colour,price,tax_band_a,tax_band_b,tax_band_c,tax_band_d,tax_band_e,type_coupe,type_estate,type_hatchback,type_saloon,type_suv,make_model,make_colour,make_model_colour,year_county
654,-1.254703,-0.411111,-0.126048,-0.071977,0.968045,41920.0,1,1,2,1,1,1,1,1,1,2,-0.184799,0.517827,0.245147,-0.411933
1947,-1.254703,2.018788,-0.638329,0.728682,0.968045,34961.0,1,2,1,1,1,1,1,1,1,2,-0.072122,0.009563,0.376349,2.016761
3698,1.195070,-0.411111,-1.150611,-1.558914,-1.661267,69784.0,1,2,1,1,1,1,1,1,2,1,-1.236449,-1.311922,-1.049376,-0.410328


In [52]:
lr_scaler_model = LinearRegression()
lr_scaler_model.fit(X=X_train, y=y_train)
lr_scaler_model_prodict = lr_scaler_model.predict(X=X_test)
lr_scaler_model_mse = mean_squared_error(y_pred=lr_scaler_model_prodict, y_true=y_test)
lr_scaler_model_r_sq = r2_score(y_pred=lr_scaler_model_prodict, y_true=y_test)

###Valitation Test:: Linear Regressor Model Standard Scaler and Best Features Dummies

In [53]:
print(f'Linear Model StandardScaler Features Accuracy:\nRMSE={math.sqrt(lr_scaler_model_mse):,.0f}\nRSq={lr_scaler_model_r_sq:.2f} or {lr_scaler_model_r_sq*100:.0f}% of the variability in Y can be explained using X')

Linear Model StandardScaler Features Accuracy:
RMSE=27,689
RSq=0.14 or 14% of the variability in Y can be explained using X


In [54]:
lr_scaler_model.coef_

array([[ 1.03344360e+02, -1.32232678e+06, -5.97751995e+03,
        -4.09959472e+03, -1.38655738e+03, -3.99376401e+03,
        -6.60908489e+02,  1.73377915e+03,  1.32251919e+06]])

In [55]:
lr_scaler_model.score

<bound method RegressorMixin.score of LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)>

In [56]:
lr_scaler_model.intercept_, lr_scaler_model.rank_

(array([46814.18519737]), 9)

In [ ]:
#pickle.dump(lr_scaler_model, open('/content/drive/MyDrive/Data Science and Analytics/R00206995/model/model.pkl', 'wb') )

In [ ]:
shapiro_test = stats.shapiro(X_train)
shapiro_test

/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:1676: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


(0.873183012008667, 0.0)